In [11]:
import pandas as pd
import time
import gc
import sklearn
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
import xgboost as xgb
from scipy.stats import uniform
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform
import ast
from sklearn.metrics import accuracy_score
random_state = 42
np.random.seed(random_state)
import xgboost as xgb
from sklearn.metrics import roc_auc_score

In [12]:
ctr_15 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_15.csv')
ctr_16 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_16.csv') 
data = pd.concat(([ctr_15,ctr_16]))
print("Tamaño del conjunto ", data.shape)
ctr_test = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_test.csv')
print("Tamaño del conjunto ", ctr_test.shape)

KeyboardInterrupt: 

In [3]:
# Lista de columnas a eliminar
columns_to_remove = [
    'device_id', 'action_list_1', 'action_list_2', 'action_categorical_0', 
    'auction_list_0', 'action_categorical_2', 'action_categorical_4', 
    'auction_categorical_0', 'auction_categorical_11', 'auction_categorical_3', 
    'creative_categorical_0', 'auction_categorical_7', 'Auction_time', 'auction_categorical_10', 'action_categorical_7', 'action_categorical_6',
    'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10',
    'auction_categorical_4', 'gender', 'creative_categorical_1', 
    'auction_categorical_6', 'auction_boolean_0', 'creative_categorical_3',
    'auction_categorical_2', 'action_categorical_5', 'action_list_0',
    'auction_categorical_1', 'auction_categorical_5', 'device_id_type',
    'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0',
    'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5',
    'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8',
    'creative_categorical_4', 'creative_categorical_6', 'action_categorical_1',
    'creative_categorical_2', 'action_categorical_3', 'action_categorical_2',
    'auction_categorical_11', 'action_categorical_4',
    'creative_categorical_0', 'device_id'
]

data = data.drop(columns=columns_to_remove, errors='ignore')
ctr_test = ctr_test.drop(columns=columns_to_remove, errors='ignore')

id_col_test = ctr_test["id"]
ctr_test = ctr_test.drop(columns="id", errors='ignore')

print(f"Columnas eliminadas: {columns_to_remove}")
print(f"Tamaño del data después de la eliminación: {data.shape}")
print(f"Tamaño del test después de la eliminación: {ctr_test.shape}")


Columnas eliminadas: ['device_id', 'auction_categorical_11', 'action_list_1', 'action_list_2', 'auction_list_0', 'action_list_0', 'creative_categorical_0', 'Auction_time', 'auction_categorical_10', 'action_categorical_7', 'action_categorical_6', 'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10', 'auction_categorical_4', 'gender', 'creative_categorical_1', 'auction_categorical_6', 'auction_boolean_0', 'creative_categorical_3', 'auction_categorical_2', 'action_categorical_5', 'action_list_0', 'auction_categorical_1', 'auction_categorical_5', 'device_id_type', 'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0', 'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5', 'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8', 'creative_categorical_4', 'creative_categorical_6', 'creative_categorical_2', 'action_categorical_2', 'auction_categorical_11', 'device_id']
Tamaño del data después de la eliminación: (2

In [4]:
def group_rare_categories(column, threshold):
    freq = column.value_counts(normalize=True)
    rare_categories = freq[freq < threshold].index
    return column.apply(lambda x: 'Other' if x in rare_categories else x)

threshold = 0.01

categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
categorical_columns_eval = ctr_test.select_dtypes(include=['object']).columns.tolist()

#categorical_columns = ['auction_categorical_12', 'auction_categorical_3', 'auction_categorical_7', 'auction_categorical_9']
#list_columns = ['action_list_2', 'action_list_1', 'auction_list_0']

for col in categorical_columns:
    data[col] = group_rare_categories(data[col], threshold)

for col in categorical_columns_eval:
    ctr_test[col] = group_rare_categories(ctr_test[col], threshold)
pd_ohe = pd.get_dummies(data,
                        columns=categorical_columns + ['has_video'],  # Incluye has_video también
                        sparse=False,    # Devolver una matriz no rala para facilitar la alineación
                        dummy_na=False,  # No agregar columna para NaNs.
                        dtype=int        # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                       )

pd_ohe_eval = pd.get_dummies(ctr_test,
                             columns=categorical_columns_eval + ['has_video'],  # Incluye has_video también
                             sparse=False,    # Devolver una matriz no rala
                             dummy_na=False,  # No agregar columna para NaNs.
                             dtype=int        # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                            )

pd_ohe_eval = pd_ohe_eval.reindex(columns=pd_ohe.columns, fill_value=0)

pd_ohe_eval['id'] = id_col_test.values

print("Shape de data_drop después de OneHotEncoding:", pd_ohe.shape)
print("Shape de eval_data_drop después de OneHotEncoding:", pd_ohe_eval.shape)

pd_ohe.info()
pd_ohe_eval.info()

Shape de data_drop después de OneHotEncoding: (2331937, 96)
Shape de eval_data_drop después de OneHotEncoding: (1139639, 97)
<class 'pandas.core.frame.DataFrame'>
Index: 2331937 entries, 0 to 1092412
Data columns (total 96 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Label                            int64  
 1   auction_age                      float64
 2   auction_bidfloor                 float64
 3   auction_time                     float64
 4   creative_height                  float64
 5   creative_width                   float64
 6   timezone_offset                  float64
 7   action_categorical_1_0d132da2    int64  
 8   action_categorical_1_3074db21    int64  
 9   action_categorical_1_62f19448    int64  
 10  action_categorical_1_6d1ca31b    int64  
 11  action_categorical_1_Other       int64  
 12  action_categorical_1_ac0f362d    int64  
 13  action_categorical_1_d9eb38d8    int64  
 14  action_categorical_1_dc24b

In [5]:
y = pd_ohe[['Label']].copy() # Usamos copy para no modificar un view abajo, ya que genera un warning.
X = pd_ohe.drop('Label', axis = 1)

X_train, X_val, Y_train, Y_val = train_test_split(X, y,
                                                  train_size = 0.8,
                                                  random_state = 32,
                                                  stratify = y)

X_test = pd_ohe_eval.drop('Label', axis = 1)

print(X_test.shape)
print(f'Cantidad de datos de train: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')


(1139639, 96)
Cantidad de datos de train: 1865549
Cantidad de datos de validación: 466388


In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Definir los hiperparámetros a buscar
param_dist = {
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(100, 300),
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5)
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb.XGBClassifier(
        objective='binary:logistic',
        seed=42,
        eval_metric='auc',
        use_label_encoder=False
    ),
    param_distributions=param_dist,
    n_iter=100,  # Número de combinaciones aleatorias a probar
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1  # Usa todos los núcleos de CPU disponibles
)

# Ajustar RandomizedSearchCV
random_search.fit(X_train, Y_train)

# Ver los mejores parámetros
print("Mejores parámetros encontrados: ", random_search.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:04:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:04:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


KeyboardInterrupt: 

In [6]:
# Define the XGBoost classifier
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                            eval_metric='auc',  # You can also use 'logloss' for binary classification
                            use_label_encoder=False)
clf_xgb.fit(X_train, Y_train, verbose = True, eval_set = [(X_val, Y_val)])

/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:39:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.79678
[1]	validation_0-auc:0.79740
[2]	validation_0-auc:0.79973
[3]	validation_0-auc:0.80152
[4]	validation_0-auc:0.80188
[5]	validation_0-auc:0.80274
[6]	validation_0-auc:0.80814
[7]	validation_0-auc:0.81168
[8]	validation_0-auc:0.81228
[9]	validation_0-auc:0.82484
[10]	validation_0-auc:0.82693
[11]	validation_0-auc:0.83023
[12]	validation_0-auc:0.83151
[13]	validation_0-auc:0.83227
[14]	validation_0-auc:0.83298
[15]	validation_0-auc:0.83352
[16]	validation_0-auc:0.83383
[17]	validation_0-auc:0.83443
[18]	validation_0-auc:0.83463
[19]	validation_0-auc:0.83478
[20]	validation_0-auc:0.83513
[21]	validation_0-auc:0.83522
[22]	validation_0-auc:0.83613
[23]	validation_0-auc:0.83624
[24]	validation_0-auc:0.83648
[25]	validation_0-auc:0.83664
[26]	validation_0-auc:0.83668
[27]	validation_0-auc:0.83657
[28]	validation_0-auc:0.83666
[29]	validation_0-auc:0.83663
[30]	validation_0-auc:0.83742
[31]	validation_0-auc:0.83748
[32]	validation_0-auc:0.83773
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [7]:
X_test['id'] = range(1, len(X_test) + 1)
y_pred_test = clf_xgb.predict_proba(X_test.drop(columns=["id"]))[:, clf_xgb.classes_ == 1].squeeze()
submission_df = pd.DataFrame({"id": X_test["id"], "Label": y_pred_test})
submission_df["id"] = submission_df["id"].astype(int)
submission_df.to_csv("XGBOOTS.csv", sep=",", index=False)